In [8]:
import torch
import pyro
import pyro.distributions as p
from pyro import infer, optim
from pyro.infer import mcmc
import random
import matplotlib.pyplot as plt

Lets generate some simple data first <br>
I just generate some coinflips from a biased coin

In [16]:
def generate_flips(n, prob=[0.8]):
    flips = p.Bernoulli(torch.tensor(prob)).expand([n]).sample()
    return flips

flips = generate_flips(100)

Now lets make an easy generative model for the coin flips where <br>
we assume a normal prior on the bernouilli distribution

In [17]:
def model(data=None):
    # we initially assume 0.5 and 1 and this is only used for initialization and 
    # overwritten by the guide later
    bernouilli_param = pyro.sample('bernouilli', p.Normal(0.5, 1))
    
    # now we generate the coinflips and compare 
    # the plate gives us conditionally independence what we ofcourse want
    # since the flips should be conditionally independence given the bernouilli value
    for i in pyro.plate('flips', len(data)):
        # sample a flip
        pyro.sample(f'flip{i}', p.Bernoulli(bernouilli_param), obs=data[i])
    
        

In [18]:
hmc_kernel = mcmc.HMC(model, step_size=0.0855, num_steps=4)
mcmc_run = mcmc.MCMC(hmc_kernel, num_samples=1000, warmup_steps=300).run(flips)
posterior = mcmc_run.marginal('bernouilli').empirical['bernouilli']
posterior.mean 

Sample: 100%|██████████| 1300/1300 [00:59<00:00, 21.86it/s, step size=1.29e+00, acc. rate=0.875]


tensor(0.8404)

In [21]:
nuts_kernel = mcmc.NUTS(model, adapt_step_size=True)
mcmc_run = mcmc.MCMC(nuts_kernel, num_samples=1000, warmup_steps=300).run(flips)
posterior = mcmc_run.marginal('bernouilli').empirical['bernouilli']
posterior.mean 

Sample: 100%|██████████| 1300/1300 [01:43<00:00, 12.60it/s, step size=1.15e+00, acc. rate=0.876]


tensor(0.8418)